# 4. 序列和反序列化

## 4.1 概念

**序列化**

在程序运行的过程中，所有的变量都是在内存中，比如，定义一个dict：

d = dict(name='Bob', age=20, score=88)

可以随时修改变量，比如把name改成'Bill'，但是一旦程序结束，变量所占用的内存就被操作系统全部回收。如果没有把修改后的'Bill'存储到磁盘上，下次重新运行程序，变量又被初始化为'Bob'。

我们把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling，在其他语言中也被称之为serialization，marshalling，flattening等等，都是一个意思。

序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上。

换句话，把变量或对象从内存中变成可存储或传输的过程称之为序列化

**反序列化**

反过来，把变量内容从序列化的对象重新读到内存里称之为反序列化，即unpickling。

在Python中，有两种提供序列化的模块，一个是pickle，一个是json，

前者，序列化变量或对象成字节流，但却是Python独有的模块，序列化后的字节流，无法和其他语言进行交互，

后者，可将对象序列化成网络中标准的数据传输格式---->json串，这是一个轻量级的数据交互格式，相对于XML来说，简洁且高效。

## 4.2 pickle, cPickle 模块：序列化 Python 对象

`pickle` 模块实现了一种算法，可以将任意一个 `Python` 对象转化为一系列的字节，也可以将这些字节重构为一个有相同特征的新对象。

由于字节可以被传输或者存储，因此 `pickle` 事实上实现了传递或者保存 `Python` 对象的功能。

`cPickle` 使用 `C` 而不是 `Python` 实现了相同的算法，因此速度上要比 `pickle` 快一些。但是它不允许用户从 `pickle` 派生子类。如果子类对你的使用来说无关紧要，那么 `cPickle` 是个更好的选择。

In [4]:
try:
    import cPickle as pickle
except:
    import pickle

pickle模块提供了四个功能：dumps、dump、loads、load。（json模块和pickle提供的一样，但是使用起来还是稍微有点差别）

    序列化

    dumps ： 直接将变量或对象序列化成bytes，可以用print打印内容
    dump   ： 将变量或对象序列化后的内容写入指定的文件

    反序列化

    loads  ： 直接将序列化后的字节流，反序列化（加载，还原）成序列化之前的状态，可以用print打印内容
    load    ： 将一个序列化后的文件反序列化（加载，还原）成变量或者对象

## 4.3 传输模式 dumps和loads

### 4.3.1 本地反序列化

使用 `pickle.dumps()` 可以将一个对象转换为字符串（`dump string`）：

In [5]:
data = [ { 'a':'A', 'b':2, 'c':3.0 } ]

data_string = pickle.dumps(data)  # 变量data_string和之前的变量data是两个毫不相干的对象，但是，他们的内容都是一样的

print("DATA:")
print(data)
print("PICKLE:")
print(data_string)

DATA:
[{'a': 'A', 'b': 2, 'c': 3.0}]
PICKLE:
b'\x80\x03]q\x00}q\x01(X\x01\x00\x00\x00aq\x02X\x01\x00\x00\x00Aq\x03X\x01\x00\x00\x00bq\x04K\x02X\x01\x00\x00\x00cq\x05G@\x08\x00\x00\x00\x00\x00\x00ua.'


虽然 `pickle` 编码的字符串并不一定可读，但是我们可以用 `pickle.loads()` 来从这个字符串中恢复原对象中的内容（`load string`）：

In [3]:
data_from_string = pickle.loads(data_string)

print(data_from_string)

[{'a': 'A', 'b': 2, 'c': 3.0}]


### 4.3.2 编码协议

`dumps` 可以接受一个可省略的 `protocol` 参数（默认为 0），目前有 3 种编码方式：

- 0：原始的 `ASCII` 编码格式
- 1：二进制编码格式
- 2：更有效的二进制编码格式

当前最高级的编码可以通过 `HIGHEST_PROTOCOL` 查看：

In [4]:
print(pickle.HIGHEST_PROTOCOL)

4


例如：

In [5]:
data_string_1 = pickle.dumps(data, 1)

print("Pickle 1:", data_string_1)

data_string_2 = pickle.dumps(data, 2)

print("Pickle 2:", data_string_2)

Pickle 1: b']q\x00}q\x01(X\x01\x00\x00\x00aq\x02X\x01\x00\x00\x00Aq\x03X\x01\x00\x00\x00bq\x04K\x02X\x01\x00\x00\x00cq\x05G@\x08\x00\x00\x00\x00\x00\x00ua.'
Pickle 2: b'\x80\x02]q\x00}q\x01(X\x01\x00\x00\x00aq\x02X\x01\x00\x00\x00Aq\x03X\x01\x00\x00\x00bq\x04K\x02X\x01\x00\x00\x00cq\x05G@\x08\x00\x00\x00\x00\x00\x00ua.'


如果 `protocol` 参数指定为负数，那么将调用当前的最高级的编码协议进行编码：

In [6]:
print(pickle.dumps(data, -1))

b'\x80\x04\x95#\x00\x00\x00\x00\x00\x00\x00]\x94}\x94(\x8c\x01a\x94\x8c\x01A\x94\x8c\x01b\x94K\x02\x8c\x01c\x94G@\x08\x00\x00\x00\x00\x00\x00ua.'


从这些格式中恢复对象时，不需要指定所用的协议，`pickle.loads()` 会自动识别：

In [7]:
print("Load 1:", pickle.loads(data_string_1))
print("Load 2:", pickle.loads(data_string_2))

Load 1: [{'a': 'A', 'b': 2, 'c': 3.0}]
Load 2: [{'a': 'A', 'b': 2, 'c': 3.0}]


### 4.3.3 网络传输反序列化

**服务端**

In [ ]:
#!/usr/bin/env Python3  
# -*- coding:utf-8 -*-  
  
import socket ,pickle  
  
class Student(object):  # 一定要声明下这个Student类，否则反序列化出对象的时候，找不到类标志  
    pass  

serSock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)  
serSock.bind(('0.0.0.0',9999))  
serSock.listen(5)  
print('等待连接......')  
s,addr = serSock.accept()  
  
result = s.recv(1024)    #来至于客户端的 消息  
  
print('客户端：',result)      
a = pickle.loads(result) #反序列化   

print(stu.name,stu.sex,stu.age)  

s.send(bytes('服务端：消息接收完毕！',encoding='utf-8'))  
serSock.close           #最后别忘关了套接字对象，关闭资源  

等待连接......


**客户端**

In [ ]:
!/usr/bin/env Python3  
# -*- coding:utf-8 -*-  
import pickle,socket  
  
    
class Student(object):  
    
    def __init__(self,name,sex,age):  
        self.name = name  
        self.age  = age  
        self.sex  = sex  
    def show(self):  
        print(self.name,self.sex,self.age)  
        
        
s = Student('appleyk','男',26)  
byte = pickle.dumps(s)        #序列化实例s  --->字节流   
stu  = pickle.loads(byte)  
#stu.show() #注意，方法是无法通过传输，进行反序列化的，感兴趣的可以自己验证下  
  
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)  
s.connect(('127.0.0.1',9999))    
s.send(byte)  
print(s.recv(1024).decode('utf-8'))  
s.close()  

我们在将序列化应用到网络传输的时候，pickle只能保存一些基本数据类型（这就够用了,毕竟我们主要是传数据的，不是传递行为的），因此，如果一些对象中的成员，传输后，无法被成功的反序列化，请不要大惊小怪，毕竟，对象的行为操作，是有域的，如果离开了自身所处的环境，其他地方是无法正常运转的，就好比，淡水中的鱼，我们可以拿来享用，但是，如果，我们把它放在咸水中饲养一段时间后再享用，肯定是不行的！它会挂掉的。

## 4.4 文件模式（存储和读取 ） dump和load

除了将对象转换为字符串这种方式，`pickle` 还支持将对象写入一个文件中，通常我们将这个文件命名为 `xxx.pkl`，以表示它是一个 `pickle` 文件： 

存储和读取的函数分别为：

- `pickle.dump(obj, file, protocol=0)` 将对象序列化并存入 `file` 文件中
- `pickle.load(file)` 从 `file` 文件中的内容恢复对象

将对象存入文件：

In [20]:
with open("data.pkl", "wb") as f:
    pickle.dump(data, f)

从文件中读取：

In [30]:
# pickle.load?
with open("data.pkl",  'rb') as f:
    data_from_file = pickle.load(f)
    
print(data_from_file)

[{'a': 'A', 'b': 2, 'c': 3.0}]


清理生成的文件：

In [31]:
import os
os.remove("data.pkl")